```
4. Preprocessing Pipeline
The partner must:
describe the data preprocessing pipeline, and 
how this is accomplished via a package/function that is a callable API (that is ultimately accessed by the served, production model).

Evidence must include a description (in the Whitepaper) of how data preprocessing is accomplished, along with the code snippet that accomplishes data preprocessing as a callable API.
```

# Deploy model using TFX Pipeline
To deploy the model we will following GCP best practises and use TensorFlow Extended (TFX). A TensorFlow pipeline is a sequence of components taht impoement an ML Pipeline which is specificially designed for scale, deployment, and retraining.

To successfully deploy the model we will need address the 3 phases of the pipeline:
1. Ingest & Validate Data
  - ExampleGen
  - StatisticsGen
  - SchemaGen
  - ExampleValidator
2. Train & Analyze Model
  - Transform
  - Trainer
3. Deploy in Production
  - Pusher

### Install python packages
We will install required Python packages including TFX and KFP to author ML pipelines and submit jobs to Vertex Pipelines. We will be deploying the TFX pipeline onto the Apache Beam orchistrator.

In [ ]:
# Use the latest version of pip.
!pip install --upgrade pip
!pip install --upgrade "tfx[kfp]<2"
!pip install --upgrade tensorflow_transform

  Using cached tensorflow-2.7.1-cp37-cp37m-manylinux2010_x86_64.whl (495.0 MB)
  Using cached tfx_bsl-1.6.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (19.1 MB)
  Using cached tensorflow_transform-1.6.0-py3-none-any.whl (427 kB)
  Using cached keras-2.7.0-py2.py3-none-any.whl (1.3 MB)
  Using cached tensorflow_metadata-1.6.0-py3-none-any.whl (48 kB)
  Using cached tensorflow_serving_api-2.7.0-py2.py3-none-any.whl (37 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: tensorflow-metadata
    Found existing installation: tensorflow-metadata 1.7.0
    Uninstalling tensorflow-metadata-1.7.0:
      Successfully uninstalled tensorflow-metadata-1.7.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.0
    Uninstalling tensorflow-2.8.0:
      Successfully uninstalled tensorflow-2.8.0
  Attempting uninstall: tensorflow-servi

### Restart the Runtime
You will need to restart the runtime for the libraries to be available in Google Collab. Runtime > Restart Runtime

### Login in to Google for this *notebook*

In [ ]:
import sys
if 'google.colab' in sys.modules:
  from google.colab import auth
  auth.authenticate_user()

### Check the package versions.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))
import kfp
print('KFP version: {}'.format(kfp.__version__))

TensorFlow version: 2.8.0
TFX version: 1.6.1
KFP version: 1.8.11


### Set up variables

We will set up some variables used to customize the pipelines below. Following
information is required:

* GCP Project id.
* GCP Region to run pipelines.
* Google Cloud Storage Bucket to store pipeline outputs.

In [ ]:
GOOGLE_CLOUD_PROJECT = 'ml-spec-demo-2-sandbox'
GOOGLE_CLOUD_REGION = 'australia-southeast1'
GCS_BUCKET_NAME = 'black_friday_gcp_bucket'

#### Set `gcloud` to use your project.

In [ ]:
 {GOOGLE_CLOUD_PROJECT}

Updated property [core/project].


### Set up Global variables for model serving locations

In [ ]:
PIPELINE_NAME = 'black-friday-gcp-vertex-pipelines'

# Path to pipeline artifacts
PIPELINE_ROOT = 'gs://{}/pipeline_root/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths for users' Python module
MODULE_ROOT = 'gs://{}/pipeline_module/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)

# Paths to training data
DATA_ROOT = 'gs://{}/data/{}'.format(GCS_BUCKET_NAME, PIPELINE_NAME)

# This is the path where your model will be pushed for serving
SERVING_MODEL_DIR = 'gs://{}/serving_model/{}'.format(
    GCS_BUCKET_NAME, PIPELINE_NAME)


# Training data file name
FILE_NAME = 'train.csv'

print('PIPELINE_ROOT: {}'.format(PIPELINE_ROOT))
print('Data root: {}'.format(DATA_ROOT))

PIPELINE_ROOT: gs://black_friday_gcp_bucket/pipeline_root/black-friday-gcp-vertex-pipelines
Data root: gs://black_friday_gcp_bucket/data/black-friday-gcp-vertex-pipelines


We need to make our own copy of the dataset. Because TFX ExampleGen reads
inputs from a directory, we need to create a directory and copy dataset to it
on GCS.

Take a quick look at the CSV file.

In [ ]:
!gsutil cat {DATA_ROOT}/train.csv | head

User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
1000001,P00069042,F,0-17,10,A,2,0,3,,,8370
1000001,P00248942,F,0-17,10,A,2,0,1,6,14,15200
1000001,P00087842,F,0-17,10,A,2,0,12,,,1422
1000001,P00085442,F,0-17,10,A,2,0,12,14,,1057
1000002,P00285442,M,55+,16,C,4+,0,8,,,7969
1000003,P00193542,M,26-35,15,A,3,0,1,2,,15227
1000004,P00184942,M,46-50,7,B,2,1,1,8,17,19215
1000004,P00346142,M,46-50,7,B,2,1,1,15,,15854
1000004,P0097242,M,46-50,7,B,2,1,1,16,,15686


## Create a pipeline

TFX pipelines are defined using Python APIs. We will define a pipeline which
consists of three components, CsvExampleGen, Trainer and Pusher. The pipeline
and model definition is almost the same as
[Simple TFX Pipeline Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

The only difference is that we don't need to set `metadata_connection_config`
which is used to locate
[ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) database. Because
Vertex Pipelines uses a managed metadata service, users don't need to care
of it, and we don't need to specify the parameter.

Before actually define the pipeline, we need to write a model code for the
Trainer component first.

# Write Example Component


### Write model code.

We will use the same model code as in the
[Simple TFX Pipeline Tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple).

In [ ]:
_trainer_module_file = 'bfs_trainer.py'
_transformer_module_file = 'transformer.py'
_training_pipeline_file = 'training_pipeline.py'

In [ ]:
%%writefile {_transformer_module_file}

from typing import Dict, Text, Any, List
import tensorflow as tf
import tensorflow_transform as tft

FEATURES = [
            'Product_ID',
            'Gender',
            'Age',
            'Occupation',
            'City_Category', 
            'Stay_In_Current_City_Years',
            'Marital_Status',
            'Product_Category_1',
            'Product_Category_2',
            'Purchase'
            ]

CATEGORICAL_FEATURE_KEYS = [
                            'Product_ID', 
                            'Age', 
                            'City_Category', 
                            'Product_Category_1', 
                            'Product_Category_2', 
                            'Stay_In_Current_City_Years',
                            'Gender'
                            ]

OPTIONAL_NUMERIC_KEY_FEATURES = ['Product_Category_2']

def preprocessing_fn(inputs: Dict[Text, Any], custom_config) -> Dict[Text, Any]:
    """tf.transform's callback function for preprocessing inputs.
    Args:
      inputs: map from feature keys to raw not-yet-transformed features.
      custom_config:
        timesteps: The number of timesteps in the look back window
        features: Which of the features from the TF.Example to use in the model.
    Returns:
      Map from string feature key to transformed feature operations.
    """
    print('Start preprocessing')
    outputs = {}
    for key in FEATURES:
      key_l = key.lower()
      outputs[key_l] = inputs[key]


    # Convert optional categories to sparse tensor (fills in blank values basically)
    for key in OPTIONAL_NUMERIC_KEY_FEATURES:
      key_l = key.lower()
      sparse = tf.sparse.SparseTensor(inputs[key].indices, inputs[key].values,
                                      [inputs[key].dense_shape[0], 1])
      dense = tf.sparse.to_dense(sp_input=sparse, default_value=0)

      # Reshaping from a batch of vectors of size 1 to a batch to scalars.
      dense = tf.squeeze(dense, axis=1)
      outputs[key_l] = dense

    for key in CATEGORICAL_FEATURE_KEYS:
      key_l = key.lower()
      outputs[key_l] = tft.compute_and_apply_vocabulary(inputs[key])

    return outputs

Writing transformer.py


In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow.keras import layers
from tfx_bsl.tfxio import dataset_options
from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import tensorflow_transform as tft
from tfx.components.example_gen import utils as example_gen_utils

from tensorflow_metadata.proto.v0 import schema_pb2

FEATURES = [
            'product_id',
            'gender',
            'age', 
            'occupation', 
            'city_category', 
            'stay_in_current_city_years', 
            'marital_status', 
            'product_category_1',
            'product_category_2'
            ]

LABEL = 'purchase'

# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer_inference = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    """Returns the output to be used in the serving signature."""
    # raw_feature_spec = tf_transform_output.raw_feature_spec()
    # Remove label feature since these will not be present at serving time.
    # raw_feature_spec.pop(LABEL)
    # logging.info(f"Raw_Feature_Spec: {raw_feature_spec}")
    # raw_features = tf.io.parse_example(serialized_tf_examples, raw_feature_spec)
    #json_dict = json.loads(raw_features) # json to dict
    #parsed_features = example_gen_utils.dict_to_example(json_dict) # TFX contains a utility fn for converting a dict to (unserialized) tf Example 
    #transformed_features = model.tft_layer_inference(parsed_features)
    feature_spec = tf_transform_output.raw_feature_spec()
    feature_spec.pop("Purchase")
    feature_spec.pop("User_ID")
    parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
    transformed_features = model.tft_layer_inference(parsed_features)
    outputs = model(transformed_features)
    return {'outputs': outputs}

  return serve_tf_examples_fn


def _make_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in FEATURES]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(128, activation='relu')(d)
  d = keras.layers.Dense(256, activation='relu')(d)
  d = keras.layers.Dense(128, activation='relu')(d)
  outputs = keras.layers.Dense(1)(d)

  model = tf.keras.Model(inputs=inputs, outputs=outputs)

  model.compile(
      optimizer=tf.optimizers.Adam(learning_rate=0.0005), 
      loss=tf.keras.losses.MeanSquaredError(),
      metrics=[keras.metrics.MeanSquaredError()]
    )
  
  model.summary(print_fn=logging.info)

  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  
  # get transform component output
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  # read input data
  train_dataset = fn_args.data_accessor.tf_dataset_factory(
      fn_args.train_files,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=20,
          label_key=LABEL
      ),
      tf_transform_output.transformed_metadata.schema,
  )

  eval_dataset = fn_args.data_accessor.tf_dataset_factory(
      fn_args.eval_files,
      dataset_options.TensorFlowDatasetOptions(
          batch_size=10,
          label_key=LABEL
      ),
      tf_transform_output.transformed_metadata.schema,
  )

  model = _make_keras_model()


  # Train model
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps
  )

  # The layer has to be saved to the model for keras tracking purpases.
  model.tft_layer = tf_transform_output.transform_features_layer()
  
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output)
  }



  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Overwriting bfs_trainer.py


### Copy files to bucket
The transform and trainer module files need to be copied over to the GCP bucket for TFX to read.

In [ ]:
!gsutil cp {_trainer_module_file} {MODULE_ROOT}/
!gsutil cp {_transformer_module_file} {MODULE_ROOT}/

Copying file://bfs_trainer.py [Content-Type=text/x-python]...
/ [1 files][  4.8 KiB/  4.8 KiB]                                                
Operation completed over 1 objects/4.8 KiB.                                      
Copying file://transformer.py [Content-Type=text/x-python]...
-
Operation completed over 1 objects/2.1 KiB.                                      


### Create TFX pipeline. This pipeline can then be passed onto an orchestrator, such as KubeFlow, for deployment.

In [ ]:
import os
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils
from tensorflow.keras import layers
from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tfx.orchestration.pipeline import Pipeline
from tfx.proto.trainer_pb2 import EvalArgs, TrainArgs

# docs_infra: no_execute
from google.cloud import aiplatform
from google.cloud.aiplatform import pipeline_jobs

def build_pipeline(pipeline_name, pipeline_root, serving_model_dir, data_root, file_name):
  print("Running pipeline")

  print("Creating example_gen")
  # Generate Training Samples from Dataset stored on bucket.
  example_gen = tfx.components.CsvExampleGen(
    input_base=data_root
  )

  print("Creating statistics_gen")
  # Generate statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs["examples"]
  )

  print("Creating schema_gen")
  # Generates schema based on statistic files
  schema_gen = tfx.components.SchemaGen(
      statistics=statistics_gen.outputs["statistics"],
      #infer_feature_shape=True  
  )

  print("Creating example_validator")
  # Performs anomaly dection based on statistics and data schema
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs["statistics"],
      schema=schema_gen.outputs["schema"]
  )

  print("Creating transform")
  transform = tfx.components.Transform(
      examples=example_gen.outputs["examples"],
      schema=schema_gen.outputs["schema"],
      module_file=os.path.join(MODULE_ROOT, _transformer_module_file)
  )

  print("Creating trainer")
  # Trains the model
  trainer = tfx.components.Trainer(
      examples=transform.outputs["transformed_examples"],
      transform_graph=transform.outputs["transform_graph"],
      module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
      schema=schema_gen.outputs["schema"],
      train_args=TrainArgs(num_steps=150),
      eval_args=EvalArgs(num_steps=150)

  )

  print("Creating pusher")
  # Pushes the trained model to vertex
  pusher = tfx.components.Pusher(
      trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)),
  )
  
  print("Creating tfx_pipeline")
  tfx_pipeline = Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      components=[
          example_gen,
          statistics_gen,
          schema_gen,
          example_validator,
          transform,
          trainer,
          pusher
      ],
      data_root=data_root,
      module_file=os.path.join(MODULE_ROOT, _trainer_module_file),
      serving_model_dir=serving_model_dir,
      enable_cache=False
  )


  pipeline_definition_file = pipeline_name + '_pipeline.json'

  print("Creating runner")
  runner = tfx.orchestration.experimental.KubeflowV2DagRunner(
      config=tfx.orchestration.experimental.KubeflowV2DagRunnerConfig(),
      output_filename=pipeline_definition_file)
  

  print("Executing runner")
  # Following function will write the pipeline definition to PIPELINE_DEFINITION_FILE.
  _ = runner.run(tfx_pipeline)


def deploy_to_vertex(pipeline_name, project_name, cloud_region):
  aiplatform.init(project=project_name, location=cloud_region)
  pipeline_definition_file = pipeline_name + '_pipeline.json'
  job = pipeline_jobs.PipelineJob(template_path=pipeline_definition_file,
                                display_name=pipeline_name)
  job.run(sync=False)

build_pipeline(PIPELINE_NAME, PIPELINE_ROOT, SERVING_MODEL_DIR, DATA_ROOT, FILE_NAME)

Running pipeline
Creating example_gen
Creating statistics_gen
Creating schema_gen
Creating example_validator
Creating transform
Creating trainer
Creating pusher
Creating tfx_pipeline
Creating runner
Executing runner


# Deploy Pipeline to Vertex AI
TFX can be deployed to Vertex AI. The build_pipeline function deploys the pipeline to using the Kubeflow V2 orchestrator.

In [ ]:
deploy_to_vertex(PIPELINE_NAME, GOOGLE_CLOUD_PROJECT, GOOGLE_CLOUD_REGION)

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob


### Model Accuracy
You can check the model accuracy by reading the logs outputted by trainer module in Vertex AI. The accuracy seems to match the Neural Networks done in the exploration phase coming in with a RME of 80,064,008.

### Setting Up Vertex AI Endpoints

Once the model is trained it is placed into the Google Bucket under: 

```
black_friday_gcp_bucket/serving_model/black-friday-gcp-vertex-pipelines/#
```

This model will need to be registered in Vertex AI's Model service.

Once this model has been setup, Vertex AI's endpoint service can easily be configured to point to the registered model.

### Inferencing
Once the endpoint has been set up it can be inferenced using a grpc request. 

To inference request's body structure is:

```
      {"instances": 
        [{
          "examples": {
            b64: "<base64 encoded, serialized tensorflow example>"
          }
        }]
      }
```


This request can be done using the following client code.

In [ ]:
# GCP Endpoint ID
ENDPOINT = "6519734516804747264"

In [ ]:
import tensorflow as tf
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(product_id, gender, age, occupation, city_category, stay_in_current_city_years, marital_status, product_category_1, product_category_2):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'Product_ID': _bytes_feature(product_id),
      'Gender': _bytes_feature(gender),
      'Age': _bytes_feature(age),
      'Occupation': _int64_feature(occupation),
      'City_Category': _bytes_feature(city_category),
      'Stay_In_Current_City_Years': _bytes_feature(stay_in_current_city_years),
      'Marital_Status': _int64_feature(marital_status),
      'Product_Category_1': _int64_feature(product_category_1),
      'Product_Category_2': _int64_feature(product_category_2),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

In [ ]:
packet = {
    "product_id": b'P00069042',
    "gender": b'F',
    "age": b'0-17',
    "occupation": 10,
    "city_category": b'A',
    "stay_in_current_city_years": b'2',
    "marital_status": 0,
    "product_category_1": 2,
    "product_category_2": 6
}

serialized_example = serialize_example(**packet)
print(f"Serialized_example: {serialized_example}")

Serialized_example: b"\n\x8c\x02\n\x12\n\x07User_ID\x12\x07\x1a\x05\n\x03\xc1\x84=\n\x12\n\x08Purchase\x12\x06\x1a\x04\n\x02\x88'\n\x0f\n\x03Age\x12\x08\n\x06\n\x040-17\n\x1b\n\nProduct_ID\x12\r\n\x0b\n\tP00069042\n\x1b\n\x12Product_Category_1\x12\x05\x1a\x03\n\x01\x02\n#\n\x1aStay_In_Current_City_Years\x12\x05\n\x03\n\x012\n\x1b\n\x12Product_Category_2\x12\x05\x1a\x03\n\x01\x06\n\x16\n\rCity_Category\x12\x05\n\x03\n\x01A\n\x0f\n\x06Gender\x12\x05\n\x03\n\x01F\n\x13\n\nOccupation\x12\x05\x1a\x03\n\x01\n\n\x17\n\x0eMarital_Status\x12\x05\x1a\x03\n\x01\x00"


Check that the serialized example correctly parses back.

In [ ]:
import tensorflow as tf

example_proto = tf.train.Example.FromString(serialized_example)
example_proto

features {
  feature {
    key: "Age"
    value {
      bytes_list {
        value: "0-17"
      }
    }
  }
  feature {
    key: "City_Category"
    value {
      bytes_list {
        value: "A"
      }
    }
  }
  feature {
    key: "Gender"
    value {
      bytes_list {
        value: "F"
      }
    }
  }
  feature {
    key: "Marital_Status"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "Occupation"
    value {
      int64_list {
        value: 10
      }
    }
  }
  feature {
    key: "Product_Category_1"
    value {
      int64_list {
        value: 2
      }
    }
  }
  feature {
    key: "Product_Category_2"
    value {
      int64_list {
        value: 6
      }
    }
  }
  feature {
    key: "Product_ID"
    value {
      bytes_list {
        value: "P00069042"
      }
    }
  }
  feature {
    key: "Purchase"
    value {
      int64_list {
        value: 5000
      }
    }
  }
  feature {
    key: "Stay_In_Current_City_Years"
    va

Encode packet into Base64, build packet and inference.

In [ ]:
from google.cloud import aiplatform
import base64

b64_example = base64.b64encode(serialized_example).decode("utf-8")
print(f"Base64 encoded example: {b64_example}")
instances_packet = [{
  "examples": {
    "b64": b64_example
  }
}]
aiplatform.init(project=GOOGLE_CLOUD_PROJECT, location='australia-southeast1')
endpoint = aiplatform.Endpoint(ENDPOINT)
prediction = endpoint.predict(instances=instances_packet)
print(prediction)

Base64 encoded example: CowCChIKB1VzZXJfSUQSBxoFCgPBhD0KEgoIUHVyY2hhc2USBhoECgKIJwoPCgNBZ2USCAoGCgQwLTE3ChsKClByb2R1Y3RfSUQSDQoLCglQMDAwNjkwNDIKGwoSUHJvZHVjdF9DYXRlZ29yeV8xEgUaAwoBAgojChpTdGF5X0luX0N1cnJlbnRfQ2l0eV9ZZWFycxIFCgMKATIKGwoSUHJvZHVjdF9DYXRlZ29yeV8yEgUaAwoBBgoWCg1DaXR5X0NhdGVnb3J5EgUKAwoBQQoPCgZHZW5kZXISBQoDCgFGChMKCk9jY3VwYXRpb24SBRoDCgEKChcKDk1hcml0YWxfU3RhdHVzEgUaAwoBAA==
Prediction(predictions=[[4547.0752]], deployed_model_id='1053314547223363584', explanations=None)


### Cloud Function
To get an inference from the model a query needs to be made to the cloud function with the packet structure:

```
{
  "User_ID: 1000047 #A Valid User_ID (Int)
}
```

The cloud function will then gather the top 10 Product_IDs the user is most_likely to buy, the User's profile, and Product Information before returning a summation of the Users expected spenditure for the month on these products.

This code can be found in the "Cloud Function" folder under ``` main.py ```.